## Fazendo import das dependências

In [ ]:
from simulations_functions import asyncSimulations_Linux
from BHA_functions import DataCollector
from BHA_functions import GraphicGenerator
from datetime import datetime

## Definindo as constantes das simulações

In [ ]:
CORES = 12
RUNS_PER_POINT = 1

TARGET = False
GRAPHIC_POINTS = 7

NETWORK_PROB = 0.8
TOPOLOGY = 'Grade'
INTENSITY_VARIATION = 0.1
BHA_NODES = (1, 3, 5)

## Realizando todas as simulações

In [ ]:
intensity = []
for c in range(1, GRAPHIC_POINTS+1):
    intensity.append(NETWORK_PROB - c*INTENSITY_VARIATION)

for nodes in BHA_NODES:
    for i in range(0, GRAPHIC_POINTS):
        start = datetime.now()
        simulations_params = {
            'runs':RUNS_PER_POINT,
            'topology':TOPOLOGY,
            'number_nodes':20,
            'rows':4,
            'columns':3,
            'prob_edge_creation':0.1,
            'edges_to_attach':3,
            'entanglements_replanished':10,
            'requests':100,
            'attempts_per_request':2,
            'network_prob':NETWORK_PROB,
            'num_black_holes':nodes,
            'black_hole_prob':intensity[i],
            'black_hole_target':TARGET
        }
        # Running simulation
        simulations_dc = asyncSimulations_Linux(cores=CORES, **simulations_params)
        # Saving CSV file
        simulations_dc.save(file_name=f'default_simulations/Target-{TARGET}/{nodes}nodes/point{i}')
        # BenchMark
        print(f"As {simulations_params['runs']} simulações finalizaram no tempo de: {datetime.now()-start}")
        print('-='*50)

## Coletando os dados diretamente dos CSVs de cada simulação

In [ ]:
datacollectors_list = []
for nodes in BHA_NODES:
    temp_datacollectors = []
    for i in range(0, GRAPHIC_POINTS):
        temp_dc = DataCollector()
        temp_dc.get_DataFrame_csv(f'default_simulations/Target-{TARGET}/{nodes}nodes/point{i}.csv')
        temp_datacollectors.append(temp_dc)
    datacollectors_list.append(tuple(temp_datacollectors))

## Criando uma função genérica para plotar os gráficos

In [ ]:
def plotGraphic(
        dataCollectors_list: list[tuple[DataCollector]], 
        intensity_variation: float,
        bha_nodes: tuple,
        y_column_name: str, 
        title: str, 
        x_label: str, 
        y_label: str) -> None:
    
    graphicGen = GraphicGenerator()

    for pos, datacollectors in enumerate(dataCollectors_list):
        graphicGen.add_on_plot(
            plot_label=f'{bha_nodes[pos]} nós', 
            x_column=(0.1, intensity_variation), 
            y_column_name=y_column_name, 
            y_standard_deviation=False,
            dc=datacollectors)

    graphicGen.show_plot(
        title=title,
        x_label=x_label, 
        y_label=y_label)

## Gerando o gráfico da Taxa de Sucesso

In [ ]:
plotGraphic(
    dataCollectors_list=datacollectors_list,
    intensity_variation=INTENSITY_VARIATION,
    bha_nodes=BHA_NODES,
    y_column_name="Success Tax",
    title="Diferença na Taxa de Sucesso vs Intensidade do Ataque",
    x_label='Intensidade',
    y_label='Taxa de Sucesso'
    )

## Gerando o Gráfico da Taxa de Eprs Usados

In [ ]:
plotGraphic(
    dataCollectors_list=datacollectors_list,
    intensity_variation=INTENSITY_VARIATION,
    bha_nodes=BHA_NODES,
    y_column_name="Used Eprs",
    title="Eprs Usados vs Intensidade do Ataque",
    x_label='Intensidade',
    y_label='Taxa de Eprs Usados'
)